In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Asthma"
cohort = "GSE230164"

# Input paths
in_trait_dir = "../../input/GEO/Asthma"
in_cohort_dir = "../../input/GEO/Asthma/GSE230164"

# Output paths
out_data_file = "../../output/preprocess/Asthma/GSE230164.csv"
out_gene_data_file = "../../output/preprocess/Asthma/gene_data/GSE230164.csv"
out_clinical_data_file = "../../output/preprocess/Asthma/clinical_data/GSE230164.csv"
json_path = "../../output/preprocess/Asthma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiling of asthma"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['gender: female', 'gender: male']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this is a SuperSeries about gene expression profiling of asthma
# This indicates it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary, we can see gender information is available at index 0
# There's no explicit trait (asthma) or age information in the sample characteristics
trait_row = None  # Trait information not directly available
age_row = None    # Age information not available
gender_row = 0     # Gender information is at index 0

# 2.2 Data Type Conversion
# For trait (unavailable, but defining function for completeness)
def convert_trait(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    # Binary conversion for asthma
    if 'asthma' in value or 'yes' in value or 'positive' in value or 'case' in value:
        return 1
    elif 'control' in value or 'no' in value or 'negative' in value or 'healthy' in value:
        return 0
    return None

# For age (unavailable, but defining function for completeness)
def convert_age(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    else:
        value = value.strip()
    
    # Try to convert to float for continuous age
    try:
        return float(value)
    except:
        return None

# For gender
def convert_gender(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    # Binary conversion: female=0, male=1
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip the clinical feature extraction


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Asthma/GSE230164/GSE230164-GPL10558_series_matrix.txt.gz


Gene data shape: (47235, 99)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers start with "ILMN_" which indicates they are Illumina probe IDs
# from the Illumina BeadArray platform. These are not human gene symbols but are 
# platform-specific probe IDs that need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the key columns from the gene annotation dictionary
# The gene identifiers in the gene expression data are "ILMN_" IDs which map to the "ID" column in gene_annotation
# The gene symbols are stored in the "Symbol" column in gene_annotation

# 2. Get gene mapping dataframe by extracting the two identified columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col="ID", gene_col="Symbol")
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])


Gene mapping dataframe shape: (44837, 2)
First few rows of gene mapping:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Gene expression data shape after mapping: (21440, 99)
First few gene symbols:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Create directory and save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since trait_row was determined to be None in step 2, we cannot extract clinical features
# This means the dataset lacks the necessary trait information for our analysis
print("No trait information available in this dataset.")

# Use is_final=False for validation since we lack trait information
is_usable = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False  # No trait information available
)

print("Dataset cannot be used for analysis because trait information is missing.")

Gene data shape after normalization: (20238, 99)


Normalized gene data saved to ../../output/preprocess/Asthma/gene_data/GSE230164.csv
No trait information available in this dataset.
Dataset cannot be used for analysis because trait information is missing.
